# 실장님 지시사항

안녕하세요.

IR팀에서 진행해온 SGA 업계 및 경쟁사 현황 자료를 시스템화 검토 진행 바랍니다.
이는 IR팀 입사 시 업계 파악 뿐 아니라,  주요 임직원들에게 공유 해 정보를 제공 하기 위함입니다.

기존에 수동으로 네이버 증권 정보에서 리서치 하여, 엑셀 자료에 반영 하였던 것으로
네이버 증권 정보 크롤링을 통해 작업 검토를 해보면 좋겠습니다.

추출되는 양식은 엑셀 외에도 좋으니, (파워BI / 원노트 등등)
자유로이 해당 항목이 적용 되도록 업무 진행 해 주시기 바랍니다.

경쟁사 : 실적 현황표에 기재된 회사들로 업데이트 (보안 / IT서비스)  
시가총액 / 종가 / 등락율 / 등락폭 / 공시 (전체) / 뉴스 (뉴스는 타이틀 위주로 1건)  
업계 : 경쟁사 자료 세팅 후 추후 진행 예정 하도록 하겠습니다.
업무 진행하면서 궁금한 사항은 문의 주세요.

### https://finance.naver.com/item/sise.naver?code=049470


In [57]:
import pandas as pd

## ① 찾아야 하는 기업들 딕셔너리화하기({"기업":"주식코드"})

In [58]:
company_dic = {"SGA": "049470", "SGA솔루션즈":"184230", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
               "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
               "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
               "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250",
               "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
               "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680"
               }



# company_dic = {
#     'SGA솔루션즈': '184230',
#     '시큐브': '131090',
#     '안랩': '053800',
#     '모니터랩': '434480',
#     '이스트소프트': '047560',
#     '윈스': '136540',
#     '한국정보인증': '053300',
#     '라온시큐어': '042510',
#     '파수': '150900',
#     '지란지교시큐리티': '208350',
#     '지니언스': '263860',
#     '한컴위드': '054920',
#     '드림시큐리티': '203650',
#     '케이사인': '192250',
#     "샌즈랩":"411080",
#     '이글루': '067920',
#     '파이오링크': '170790',
#     '시큐센': '232830',
#     '시큐레터': '418250',
#     'SGA': '049470',
#     '한글과컴퓨터': '030520',
#     'MDS테크': '086960',
#     '아이티센': '124500',
#     '쌍용정보통신': '010280'
# }

# 보이스아이, 이글루시큐리티, 코닉글로리,한컴MDS 없음

com_keys = list(company_dic.keys())

### 1-2 필요한 패키지(bs4 다운로드)

In [59]:
import requests as req
!pip install bs4
from bs4 import BeautifulSoup as bs


## ② 뉴스 제외한 수치(종가, 등락률, 시가총액, 전일대비, 거래량...) 크롤링하기
- 보이스아이는 비상장회사이므로 네이버 국내증시 사이트에서 확인할 수 없다
- 따라서 직접 넣어주는 방식을 택함

In [60]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}



# 결과 저장 리스트
all_data = []

# 회사별 최신 시세 데이터 크롤링
for company_name, stock_code in company_dic.items():
    # 종목의 일별 시세 URL
    sise_day_url = f"https://finance.naver.com/item/sise_day.naver?code={stock_code}"
    # 종목의 상세 시세 URL
    sise_url = f"https://finance.naver.com/item/sise.naver?code={stock_code}"

    # 종목의 뉴스 URL(안되서 폐기)
    # news_url = f"https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query={company_name}"

    # 최신 데이터 가져오기 (일별 시세)
    response = requests.get(sise_day_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 테이블 선택
    table = soup.find("table", class_="type2")
    rows = table.find_all("tr")

    # 첫 번째 유효한 데이터 추출
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 1:  # 데이터가 있는 행만 처리
            date = cols[0].get_text(strip=True)
            close_price = cols[1].get_text(strip=True)
            diff = cols[2].get_text(strip=True)
            diff = diff[2:]
            open_price = cols[3].get_text(strip=True)
            high_price = cols[4].get_text(strip=True)
            low_price = cols[5].get_text(strip=True)
            volume = cols[6].get_text(strip=True)
            break  # 첫 번째 유효한 행만 처리


    # 등락률 가져오기
    response = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    rate_tag = soup.select_one("strong#_rate > span")
    rate = rate_tag.text.strip() if rate_tag else "N/A"
    if rate[0] == "+": rate = rate[1:]


    # 현재가 가져오기
    response2 = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response2.text, "html.parser")
    current_tag = soup.select_one("#_nowVal")
    current_price = current_tag.text.strip() if current_tag else "N/A"


    # 거래대금 가져오기
    response3 = requests.get(sise_url, headers=headers)
    soup = BeautifulSoup(response3.text, "html.parser")
    trade_mon_tag = soup.select_one("#_amount")
    trade_mon = trade_mon_tag.text.strip() if trade_mon_tag else "N/A"

    # 시가총액 가져오기
    market_cap_tag = soup.select_one("em#_market_sum")
    market_cap = market_cap_tag.text.strip() if market_cap_tag else "N/A"

    # 뉴스 URL 가져오기(안되서 폐기)
    '''response = requests.get(news_url, headers=headers)
    bs_sga = BeautifulSoup(response.text, 'lxml')
    link = bs_sga.select_one('a.news_tit')  # 'a.news_tit' 선택자 사용
    link = link['href'] if link else "N/A"'''

    # 데이터 리스트에 추가
    all_data.append({
        "회사": company_name,
        # "현재가":current_price,
        "종가": close_price,
        "전일대비": diff,
        "등락률": rate,
        "시가": open_price,
        "고가": high_price,
        "저가": low_price,
        "거래량": volume,
        "거래대금":trade_mon,
        "시가총액(억)": market_cap,  # 시가총액 추가
        "날짜": date
    })

# DataFrame 생성
df = pd.DataFrame(all_data)
jasa = df.iloc[:2]
elsee = df.iloc[2:]
# its = df.iloc[17:]

# new_row = {
#     "회사": "[자사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# 보이스아이는 비상장회사이므로 네이버 국내증시 사이트에서 확인할 수 없다
# 따라서 직접 넣어주는 방식을 택함
voiceye_row = {
    "회사": "SGA임베디드*(코스닥 종목)",
    "종가": "2965",
    "전일대비": "0",
    "등락률":"0%",
    "시가": "2965",
    "고가": "2965",
    "저가": "2965",
    "거래량": "0",
    "거래대금": "0",
    "시가총액(억)": "161",
    "날짜": "-"

}

# sec_row = {
#     "회사": "[보안회사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# it_row = {
#     "회사": "[IT서비스]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

df = pd.concat([jasa, pd.DataFrame([voiceye_row]),elsee], ignore_index=True)

# CSV 저장
df.to_csv("latest_stock_prices_with_market_cap.csv", index=False, encoding="utf-8-sig")
df.to_excel("latest_stock_prices_with_market_cap.xlsx", index=False, engine="openpyxl")

In [61]:
df1 = df.copy()

In [62]:
df1

,회사,종가,전일대비,등락률,시가,고가,저가,거래량,거래대금,시가총액(억),날짜
0,SGA,327,5,1.55%,322,331,316,"289,562",93,192,2025.01.15
1,SGA솔루션즈,487,0,0.00%,487,494,482,"81,138",39,305,2025.01.15
2,SGA임베디드*(코스닥 종목),2965,0,0%,2965,2965,2965,0,0,161,-
3,안랩,"69,700","2,300",-3.19%,"72,800","72,900","69,100","120,003","8,449","7,755",2025.01.15
4,시큐브,936,24,2.63%,919,990,897,"720,878",674,399,2025.01.15
5,윈스,"11,500",70,-0.61%,"11,660","11,660","11,310","19,394",222,"1,412",2025.01.15
6,이글루,"5,250",40,-0.76%,"5,300","5,310","5,230","11,562",61,577,2025.01.15
7,한컴위드,"3,030",25,0.83%,"3,040","3,075","2,985","185,223",558,855,2025.01.15
8,네오리진,"1,017",32,-3.05%,"1,049","1,055",994,"21,195",22,218,2025.01.15
9,케이사인,"8,130",30,0.37%,"8,100","8,280","7,900","6,188",50,575,2025.01.15


## ③ 뉴스 데이터 크롤링하기
- 기존에는 해당 네이버 증권 사이트 -> 기업 검색 -> 뉴스 공시 페이지에서 가져오기 시도 -> 실패
- 네이버 메인에서 해당 기업 검색 -> 뉴스 페이지 -> 옵션 추가 -> 크롤링(그러나 관련없는 뉴스가 크롤링되는 오류 발생)
- 네이버 증권 사이트 -> 기업 검색 -> 종합 정보 페이지 하단의 뉴스에서 크롤링(성공)

In [63]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from urllib.parse import urlparse, parse_qs

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}



# 결과 저장 리스트
all_data = []

# 회사별 최신 시세 데이터 크롤링
for stock_code in company_dic.values():
  url = f"https://finance.naver.com/item/main.naver?code={stock_code}"



  # 최신 데이터 가져오기 (일별 시세)
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, "html.parser")
  first_news = soup.select_one('#content > div.section.new_bbs > div.sub_section.news_section > ul:nth-child(2) > li:nth-child(1) > span > a:nth-child(1)')
  if first_news:
  # 기사 제목과 URL 추출
    news_title = first_news.get_text()
    news_url = first_news['href']
    # URL 파싱
    parsed_url = urlparse(news_url)
    query_params = parse_qs(parsed_url.query)
    article_id = query_params.get("article_id", [None])[0]
    office_id = query_params.get("office_id", [None])[0]
    news_url = f"https://n.news.naver.com/mnews/article/{office_id}/{article_id}"
    print(f'기사 제목: {news_title}')
    print(f'기사 URL: {news_url}')
  else:
      print('뉴스 기사를 찾을 수 없습니다.')



    # 데이터 리스트에 추가
  all_data.append({

      # "현재가":current_price,
      "제목": news_title,
      "url": news_url

    })

# DataFrame 생성
df2 = pd.DataFrame(all_data)
jasa = df2.iloc[:2]
elsee = df2.iloc[2:]
# # its = df.iloc[17:]

# new_row = {
#     "회사": "[자사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

voiceye_row = {

     "제목": "보이스아이, 시각장애인 대상 코로나 19 정보전달을 위한 ‘음성변환 바코드’ 무상 제공",
     "url": "https://www.etoday.co.kr/news/view/1870936",


 }

# sec_row = {
#     "회사": "[보안회사]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

# it_row = {
#     "회사": "[IT서비스]",
#     "날짜": None,
#     "시가총액": None,
#     "종가": None,
#     "전일대비": None,
#     "등락률": None,
#     "시가": None,
#     "고가": None,
#     "저가": None,
#     "거래량": None,
#     "뉴스": None
# }

df2 = pd.concat([jasa, pd.DataFrame([voiceye_row]),elsee], ignore_index=True)

# CSV 저장
df2.to_csv("latest_news_of_companies.csv", index=False, encoding="utf-8-sig")
df2.to_excel("latest_news_of_companies.xlsx", index=False, engine="openpyxl")

기사 제목: SGA솔루션즈, 김광훈 전 CISO 영입...
기사 URL: https://n.news.naver.com/mnews/article/092/0002358755
기사 제목: [2025 위기극복] 국산보안 서바이벌 게임 시작…'판...
기사 URL: https://n.news.naver.com/mnews/article/138/0002189122
기사 제목: 尹체포 직후 '홍준표 테마주'만 떴...
기사 URL: https://n.news.naver.com/mnews/article/014/0005296083
기사 제목: 시큐브, 모바일 OTP 기능이 추가된 iGRIFFIN 3.0m ...
기사 URL: https://n.news.naver.com/mnews/article/029/0002924971
기사 제목: 윈스, 제로트러스트 시장 공략…프라이빗테크놀로지...
기사 URL: https://n.news.naver.com/mnews/article/092/0002360018
기사 제목: 이글루코퍼레이션, 'AI 보안 사업...
기사 URL: https://n.news.naver.com/mnews/article/092/0002359275
기사 제목: 한컴위드, 양자내성암호 시장 본격...
기사 URL: https://n.news.naver.com/mnews/article/648/0000032404
기사 제목: 네오리진, '에이펙스 걸스' 정식 ...
기사 URL: https://n.news.naver.com/mnews/article/347/0000184594
기사 제목: 케이사인, 주당 200원 결산배당 결정
기사 URL: https://n.news.naver.com/mnews/article/003/0013017677
기사 제목: 무신사, '뉴발 1조 신화' 최운식 대표 영입…S급 인...
기사 URL: https://n.news.naver.com/mnews/article/421/0008023866
기사 제목: [이슈플러스]크리덴셜 스터핑 대책 내놓은 사이버보

## ④ 수치 데이터 엑셀화하기

In [64]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers

# 데이터프레임 전처리 함수
def preprocess_dataframe(df):
    for column in df.columns:
        if (column == "회사") or (column == "날짜"):  # 회사 이름은 변환하지 않음
            continue
        try:
            # 퍼센트 처리: '%'가 포함된 경우
            if df[column].astype(str).str.contains('%').any():
                continue

            else:
                # 숫자 변환
                df[column] = df[column].str.replace(',', '', regex=False).astype(float)
        except Exception as e:
            print(f"Error processing column {column}: {e}")
    return df


# 데이터프레임 전처리
df1 = preprocess_dataframe(df1)
display(df1)

# 1. 서식화된 엑셀 파일 열기
formatted_file = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황3.xlsx"  # 서식화된 엑셀 파일 경로
output_file = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"  # 저장될 파일 경로

# 2. 기존 서식화된 엑셀 파일 불러오기
wb = load_workbook(formatted_file)
sheet = wb["작업용2"]  # 데이터 삽입할 시트 이름

# 3. 데이터 추가 위치 설정 (예: A3 셀부터 시작)
start_row = 2  # 데이터 시작 행
start_col = 2  # 데이터 시작 열 (1열 = 'A')

# 4. 데이터프레임에서 데이터 삽입
for row_index, row_data in enumerate(dataframe_to_rows(df1, index=False, header=True), start=start_row):
    for col_index, value in enumerate(row_data, start=start_col):
        cell = sheet.cell(row=row_index, column=col_index, value=value)

        # 숫자 데이터 처리
        if isinstance(value, (int, float)):
            cell.number_format = numbers.FORMAT_NUMBER  # 일반 숫자 형식

        # 퍼센트 데이터 처리
        elif isinstance(value, str) and "%" in value:
            try:
                # 퍼센트 값을 숫자로 변환 (예: "25%" -> 0.25)
                percent_value = float(value.strip('%'))/100
                cell.value = percent_value
                cell.number_format = numbers.FORMAT_PERCENTAGE  # 퍼센트 형식
            except ValueError:
                pass  # 변환 실패 시 문자열 그대로 저장


# 5. 엑셀 파일 저장
wb.save(output_file)
print(f"서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: {output_file}")


,회사,종가,전일대비,등락률,시가,고가,저가,거래량,거래대금,시가총액(억),날짜
0,SGA,327.0,5.0,1.55%,322.0,331.0,316.0,289562.0,93.0,192.0,2025.01.15
1,SGA솔루션즈,487.0,0.0,0.00%,487.0,494.0,482.0,81138.0,39.0,305.0,2025.01.15
2,SGA임베디드*(코스닥 종목),2965.0,0.0,0%,2965.0,2965.0,2965.0,0.0,0.0,161.0,-
3,안랩,69700.0,2300.0,-3.19%,72800.0,72900.0,69100.0,120003.0,8449.0,7755.0,2025.01.15
4,시큐브,936.0,24.0,2.63%,919.0,990.0,897.0,720878.0,674.0,399.0,2025.01.15
5,윈스,11500.0,70.0,-0.61%,11660.0,11660.0,11310.0,19394.0,222.0,1412.0,2025.01.15
6,이글루,5250.0,40.0,-0.76%,5300.0,5310.0,5230.0,11562.0,61.0,577.0,2025.01.15
7,한컴위드,3030.0,25.0,0.83%,3040.0,3075.0,2985.0,185223.0,558.0,855.0,2025.01.15
8,네오리진,1017.0,32.0,-3.05%,1049.0,1055.0,994.0,21195.0,22.0,218.0,2025.01.15
9,케이사인,8130.0,30.0,0.37%,8100.0,8280.0,7900.0,6188.0,50.0,575.0,2025.01.15


서식화된 엑셀 파일에 데이터가 추가되었습니다. 저장 파일: /content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx


## ⑤ 뉴스 데이터 엑셀화(수치+뉴스)

In [65]:
df = df2

In [66]:
# import pandas as pd
# from openpyxl import load_workbook
# from openpyxl.styles import Font

# # 기존 엑셀 파일 경로
# file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"
# # output_path = "/content/수치+뉴스데이터.xlsx"
# output_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx"



# # 엑셀 파일 불러오기
# wb = load_workbook(file_path)

# # 작업용2 시트 가져오기 (없으면 생성)
# if "경쟁사주가" in wb.sheetnames:
#     ws = wb["경쟁사주가"]
# else:
#     ws = wb.create_sheet("경쟁사주가")

# # 데이터 추가 시작 셀
# start_row = 4  # 행 번호
# start_col = 10  # N 열 (엑셀에서 N은 14번째 열)

# # 데이터프레임의 열 제목 추가
# for col_idx, column_name in enumerate(df.columns, start=start_col):
#     ws.cell(row=1, column=col_idx, value=column_name)

# # 데이터 추가
# for row_idx, row in df.iterrows():
#     for col_idx, (col_name, value) in enumerate(row.items(), start=start_col):
#         cell = ws.cell(row=row_idx + start_row, column=col_idx, value=value)
#         # URL 컬럼의 경우 하이퍼링크 설정
#         if col_name == "url":
#             ws.cell(row=row_idx + start_row, column=col_idx - 1).hyperlink = value  # 이전 열에 하이퍼링크
#             ws.cell(row=row_idx + start_row, column=col_idx - 1).font = Font(color="0000FF", underline="single")

# # 엑셀 파일 저장
# wb.save(output_path)
# print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")

In [67]:
'''import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"
output_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx"
#output_path = "/content/수치+뉴스데이터.xlsx"

# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 데이터 시작 행 번호
start_col = 10  # 데이터 시작 열 번호 (엑셀에서 N열은 14번째 열)

# 데이터프레임의 열 제목 추가
for col_idx, column_name in enumerate(df.columns, start=start_col):
    ws.cell(row=1, column=col_idx, value=column_name)

# 현재 입력할 행 번호
current_row = start_row

# 데이터 추가
for row_idx, row in df.iterrows():
    # 행 번호 계산 (엑셀의 row 번호와 매칭)
    excel_row_number = current_row

    # 7 또는 23일 경우 다음 행에 데이터를 밀어넣음
    if excel_row_number in [7, 23]:
        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동

    for col_idx, (col_name, value) in enumerate(row.items(), start=start_col):
        if col_name != "url":
          cell = ws.cell(row=current_row, column=col_idx, value=value)
        # URL 컬럼의 경우 하이퍼링크 설정
        if col_name == "url":
            ws.cell(row=current_row, column=col_idx - 1).hyperlink = value  # 이전 열에 하이퍼링크
            ws.cell(row=current_row, column=col_idx - 1).font = Font(color="0000FF", underline="single")

    # 다음 행으로 이동
    current_row += 1

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")'''



'import pandas as pd\nfrom openpyxl import load_workbook\nfrom openpyxl.styles import Font\n\n# 기존 엑셀 파일 경로\nfile_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"\noutput_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx"\n#output_path = "/content/수치+뉴스데이터.xlsx"\n\n# 엑셀 파일 불러오기\nwb = load_workbook(file_path)\n\n# 작업용2 시트 가져오기 (없으면 생성)\nif "경쟁사주가" in wb.sheetnames:\n    ws = wb["경쟁사주가"]\nelse:\n    ws = wb.create_sheet("경쟁사주가")\n\n# 데이터 추가 시작 셀\nstart_row = 4  # 데이터 시작 행 번호\nstart_col = 10  # 데이터 시작 열 번호 (엑셀에서 N열은 14번째 열)\n\n# 데이터프레임의 열 제목 추가\nfor col_idx, column_name in enumerate(df.columns, start=start_col):\n    ws.cell(row=1, column=col_idx, value=column_name)\n\n# 현재 입력할 행 번호\ncurrent_row = start_row\n\n# 데이터 추가\nfor row_idx, row in df.iterrows():\n    # 행 번호 계산 (엑셀의 row 번호와 매칭)\n    excel_row_number = current_row\n\n    # 7 또는 23일 경우 다음 행에 데이터를 밀어넣음\n    if excel_row_number in [7, 23]:\n        current_row += 1  # 현재 행 번호를 1 증가시켜 

In [68]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number.xlsx"
output_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx"

# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 데이터 시작 행 번호
start_col = 10  # 데이터 시작 열 번호 (엑셀에서 N열은 14번째 열)

# 현재 입력할 행 번호
current_row = start_row

# 데이터 추가 (열 제목을 제외하고 데이터프레임 내용을 추가)
for row_idx, row in df.iterrows():
    # 행 번호 계산 (엑셀의 row 번호와 매칭)
    excel_row_number = current_row

    # 7 또는 23일 경우 다음 행에 데이터를 밀어넣음
    if excel_row_number in [7, 27]:
        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동

    for col_idx, (col_name, value) in enumerate(row.items(), start=start_col):
        if col_name != "url":
            cell = ws.cell(row=current_row, column=col_idx, value=value)
        # URL 컬럼의 경우 하이퍼링크 설정
        if col_name == "url":
            ws.cell(row=current_row, column=col_idx - 1).hyperlink = value  # 이전 열에 하이퍼링크
            ws.cell(row=current_row, column=col_idx - 1).font = Font(color="0000FF", underline="single")

    # 다음 행으로 이동
    current_row += 1

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")


데이터프레임이 /content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx 파일에 성공적으로 추가되었습니다.


##⑥ 공시 정보 불러오기
- opendart api key = ff12d16bfd2cb5ddef36494b71c419fda68b304d

In [69]:
!pip install dart-fss

In [70]:
import dart_fss as dart
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import zipfile
import io
from bs4 import BeautifulSoup
api_key = 'ff12d16bfd2cb5ddef36494b71c419fda68b304d'
dart.set_api_key(api_key=api_key)

'ff12d16bfd2cb5ddef36494b71c419fda68b304d'

In [71]:
# 상장 기업명 크롤링
corp_list = dart.api.filings.get_corp_code()
corp_df = pd.DataFrame.from_dict(corp_list)
# corp_df = corp_df.dropna(subset = 'stock_code').sort_values('modify_date',ascending=False).reset_index(drop=True)
corp_df = pd.DataFrame.from_dict(corp_list).sort_values('modify_date', ascending=False).reset_index(drop=True)
corp_df['done_YN'] = "N"
corp_df



,corp_code,corp_name,stock_code,modify_date,done_YN
0,01882678,일성전력산업,None,20250114,N
1,00887506,지에스동해전력,None,20250114,N
2,00617749,유한정밀,None,20250114,N
3,01860010,오션스바이오,332190,20250114,N
4,00442631,태양기계,116100,20250114,N
...,...,...,...,...,...
109355,00417547,스피어헤드,None,20170630,N
109356,00274298,신한산전,None,20170630,N
109357,00330886,쏘텍코리아,None,20170630,N
109358,00309017,씨아이브레인,None,20170630,N


In [72]:
corp_df[corp_df['corp_name']=='보이스아이']

,corp_code,corp_name,stock_code,modify_date,done_YN
41220,01621183,보이스아이,None,20220721,N


In [73]:
# 주어진 company_dic
company_dic = {"SGA": "049470", "SGA솔루션즈":"184230", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
               "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
               "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
               "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250",
               "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
               "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680"
               }
# company_dic의 종목코드 리스트
target_stock_codes = list(company_dic.values())

# DataFrame에서 종목코드 필터링
filtered_df = corp_df[corp_df['stock_code'].isin(target_stock_codes)]

# 결과 출력
filtered_df

,corp_code,corp_name,stock_code,modify_date,done_YN
271,01038693,드림시큐리티,203650,20250102,N
441,00138303,쌍용정보통신,010280,20241220,N
520,00264635,에스넷,038680,20241217,N
853,00577016,수산아이앤티,050960,20241129,N
2904,00402110,시큐브,131090,20240829,N
3561,00988364,SGA솔루션즈,184230,20240801,N
3583,00351579,SGA,049470,20240801,N
7676,00110875,대신정보통신,020180,20240522,N
7750,01583652,모니터랩,434480,20240520,N
10971,00186559,콤텍시스템,031820,20240402,N


In [74]:
# company_dic의 keys에 따라 순서를 맞춘 DataFrame 생성
sorted_filtered_df = pd.DataFrame(columns=filtered_df.columns)

for key in company_dic.keys():
    stock_code = company_dic[key]
    matching_row = filtered_df[filtered_df['stock_code'] == stock_code]
    if not matching_row.empty:
        sorted_filtered_df = pd.concat([sorted_filtered_df, matching_row])

# 결과 출력
first_df = sorted_filtered_df.iloc[:2]
voice_eye = corp_df[corp_df['corp_name']=='보이스아이']
rest_df = sorted_filtered_df.iloc[2:]
sorted_filtered_df = pd.concat([first_df, voice_eye,rest_df], ignore_index=True)
sorted_filtered_df

,corp_code,corp_name,stock_code,modify_date,done_YN
0,00351579,SGA,049470,20240801,N
1,00988364,SGA솔루션즈,184230,20240801,N
2,01621183,보이스아이,None,20220721,N
3,00298270,안랩,053800,20230206,N
4,00402110,시큐브,131090,20240829,N
5,00868705,윈스,136540,20221206,N
6,00364847,이글루,067920,20221129,N
7,00363592,한컴위드,054920,20230331,N
8,00599106,네오리진,094860,20221202,N
9,01003040,케이사인,192250,20240327,N


In [75]:
'''import pandas as pd
import requests

# API 인증키와 고유번호 리스트
api_key = "ff12d16bfd2cb5ddef36494b71c419fda68b304d"  # 자신의 OpenDART API 키 입력
comp_list = list(sorted_filtered_df['corp_code'])

# 공시 데이터를 저장할 리스트
latest_disclosures = []

# OpenDART API URL
url_json = "https://opendart.fss.or.kr/api/list.json"

# 각 기업별 공시 데이터를 조회
for corp_code in comp_list:
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bgn_de': '20230101'  # 시작 날짜 설정
    }
    response = requests.get(url_json, params=params)
    res = response.json()

    # 공시 데이터가 없는 경우 처리
    if res['status'] == '013':  # 공시 데이터 없음
        print(f"{corp_code}: 공시 데이터 없음")
        latest_disclosures.append({
            'corp_code': corp_code,
            '공시 제목': '공시 없음',
            '공시 URL': ''
        })
        continue

    # 공시 데이터가 있을 경우
    if 'list' in res:
        # 가장 최신 공시 데이터 선택 (리스트의 첫 번째 요소)
        latest_disclosure = res['list'][0]
        title = latest_disclosure['report_nm']
        url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={latest_disclosure['rcept_no']}"
        latest_disclosures.append({
            'corp_code': corp_code,
            '공시 제목': title,
            '공시 URL': url
        })
    else:
        print(f"{corp_code}: 공시 리스트 없음")
        latest_disclosures.append({
            'corp_code': corp_code,
            '공시 제목': '공시 없음',
            '공시 URL': ''
        })

# 결과를 데이터프레임으로 변환
df_latest_disclosures = pd.DataFrame(latest_disclosures)

# 결과 출력 및 저장
print(df_latest_disclosures)
df_latest_disclosures.to_csv("latest_disclosures.csv", index=False, encoding="utf-8-sig")'''


'import pandas as pd\nimport requests\n\n# API 인증키와 고유번호 리스트\napi_key = "ff12d16bfd2cb5ddef36494b71c419fda68b304d"  # 자신의 OpenDART API 키 입력\ncomp_list = list(sorted_filtered_df[\'corp_code\'])\n\n# 공시 데이터를 저장할 리스트\nlatest_disclosures = []\n\n# OpenDART API URL\nurl_json = "https://opendart.fss.or.kr/api/list.json"\n\n# 각 기업별 공시 데이터를 조회\nfor corp_code in comp_list:\n    params = {\n        \'crtfc_key\': api_key,\n        \'corp_code\': corp_code,\n        \'bgn_de\': \'20230101\'  # 시작 날짜 설정\n    }\n    response = requests.get(url_json, params=params)\n    res = response.json()\n\n    # 공시 데이터가 없는 경우 처리\n    if res[\'status\'] == \'013\':  # 공시 데이터 없음\n        print(f"{corp_code}: 공시 데이터 없음")\n        latest_disclosures.append({\n            \'corp_code\': corp_code,\n            \'공시 제목\': \'공시 없음\',\n            \'공시 URL\': \'\'\n        })\n        continue\n\n    # 공시 데이터가 있을 경우\n    if \'list\' in res:\n        # 가장 최신 공시 데이터 선택 (리스트의 첫 번째 요소)\n        latest_disclosure = res[\'li

In [76]:
# df_latest_disclosures

In [77]:
# df_latest_disclosures = df_latest_disclosures[['공시 제목', '공시 URL']]


In [78]:
'''import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx"
output_path = f"/content/SGA업계 및 경쟁사 현황_{date}.xlsx"

# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 데이터 시작 행 번호
start_col = 9  # 데이터 시작 열 번호 (엑셀에서 N열은 14번째 열)

# # 데이터프레임의 열 제목 추가
# for col_idx, column_name in enumerate(df.columns, start=start_col):
#     ws.cell(row=1, column=col_idx, value=column_name)

# 현재 입력할 행 번호
current_row = start_row

# 데이터 추가
for row_idx, row in df_latest_disclosures.iterrows():
    # 행 번호 계산 (엑셀의 row 번호와 매칭)
    excel_row_number = current_row


    if excel_row_number in [7,27]:
        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동

    for col_idx, (col_name, value) in enumerate(row.items(), start=start_col):
        if col_name != "공시 URL":
          cell = ws.cell(row=current_row, column=col_idx, value=value)
        # URL 컬럼의 경우 하이퍼링크 설정
        if col_name == "공시 URL":
            ws.cell(row=current_row, column=col_idx - 1).hyperlink = value  # 이전 열에 하이퍼링크
            ws.cell(row=current_row, column=col_idx - 1).font = Font(color="0000FF", underline="single")

    # 다음 행으로 이동
    current_row += 1

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")'''

'import pandas as pd\nfrom openpyxl import load_workbook\nfrom openpyxl.styles import Font\n\n# 기존 엑셀 파일 경로\nfile_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx"\noutput_path = f"/content/SGA업계 및 경쟁사 현황_{date}.xlsx"\n\n# 엑셀 파일 불러오기\nwb = load_workbook(file_path)\n\n# 작업용2 시트 가져오기 (없으면 생성)\nif "경쟁사주가" in wb.sheetnames:\n    ws = wb["경쟁사주가"]\nelse:\n    ws = wb.create_sheet("경쟁사주가")\n\n# 데이터 추가 시작 셀\nstart_row = 4  # 데이터 시작 행 번호\nstart_col = 9  # 데이터 시작 열 번호 (엑셀에서 N열은 14번째 열)\n\n# # 데이터프레임의 열 제목 추가\n# for col_idx, column_name in enumerate(df.columns, start=start_col):\n#     ws.cell(row=1, column=col_idx, value=column_name)\n\n# 현재 입력할 행 번호\ncurrent_row = start_row\n\n# 데이터 추가\nfor row_idx, row in df_latest_disclosures.iterrows():\n    # 행 번호 계산 (엑셀의 row 번호와 매칭)\n    excel_row_number = current_row\n\n\n    if excel_row_number in [7,27]:\n        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동\n\n    for col_idx, (col_name, value) in enumerate(row.items(), st

In [79]:
import pandas as pd
import requests
from datetime import datetime

# API 인증키와 고유번호 리스트
api_key = "ff12d16bfd2cb5ddef36494b71c419fda68b304d"  # 자신의 OpenDART API 키 입력
comp_list = list(sorted_filtered_df['corp_code'])

# 공시 데이터를 저장할 리스트
latest_disclosures = []

# OpenDART API URL
url_json = "https://opendart.fss.or.kr/api/list.json"

# 오늘 날짜를 YYYYMMDD 형식으로 가져오기
today = datetime.today().strftime('%Y%m%d')
print(f"today is {today}")

# 각 기업별 공시 데이터를 조회
for corp_code in comp_list:
    params = {
        'crtfc_key': api_key,
        'corp_code': corp_code,
        'bgn_de': today  # 오늘 날짜부터 조회 시작
    }
    response = requests.get(url_json, params=params)
    res = response.json()

    # 공시 데이터가 없는 경우 처리
    if res['status'] == '013':  # 공시 데이터 없음
        print(f"{corp_code}: 공시 데이터 없음")
        latest_disclosures.append({
            'corp_code': corp_code,
            '공시 제목': '공시 없음',
            '공시 URL': ''
        })
        continue

    # 공시 데이터가 있을 경우
    if 'list' in res:
        # 가장 최신 공시 데이터 선택 (리스트의 첫 번째 요소)
        latest_disclosure = res['list'][0]
        rcept_date = latest_disclosure['rcept_dt']  # 등재 날짜
        if rcept_date == today:  # 등재 날짜가 오늘과 같은 경우에만 처리
            title = latest_disclosure['report_nm']
            url = f"https://dart.fss.or.kr/dsaf001/main.do?rcpNo={latest_disclosure['rcept_no']}"
            latest_disclosures.append({
                'corp_code': corp_code,
                '공시 제목': title,
                '공시 URL': url
            })
        else:
            print(f"{corp_code}: 오늘 등록된 공시가 아님 (등재 날짜: {rcept_date})")
    else:
        print(f"{corp_code}: 공시 리스트 없음")
        latest_disclosures.append({
            'corp_code': corp_code,
            '공시 제목': '공시 없음',
            '공시 URL': ''
        })

# 결과를 데이터프레임으로 변환
df_latest_disclosures = pd.DataFrame(latest_disclosures)

# 결과 출력 및 저장
print(df_latest_disclosures)
df_latest_disclosures.to_csv("latest_disclosures.csv", index=False, encoding="utf-8-sig")


today is 20250115
00351579: 공시 데이터 없음
00988364: 공시 데이터 없음
01621183: 공시 데이터 없음
00298270: 공시 데이터 없음
00402110: 공시 데이터 없음
00868705: 공시 데이터 없음
00364847: 공시 데이터 없음
00363592: 공시 데이터 없음
00599106: 공시 데이터 없음
00273420: 공시 데이터 없음
00255433: 공시 데이터 없음
00638487: 공시 데이터 없음
00306454: 공시 데이터 없음
01047169: 공시 데이터 없음
00577016: 공시 데이터 없음
00962223: 공시 데이터 없음
01038693: 공시 데이터 없음
01583652: 공시 데이터 없음
00492353: 공시 데이터 없음
01113499: 공시 데이터 없음
01579570: 공시 데이터 없음
00238782: 공시 데이터 없음
00204262: 공시 데이터 없음
00445841: 공시 데이터 없음
00620868: 공시 데이터 없음
00186559: 공시 데이터 없음
00204226: 공시 데이터 없음
00110875: 공시 데이터 없음
00138303: 공시 데이터 없음
00264635: 공시 데이터 없음
   corp_code                          공시 제목  \
0   00351579                          공시 없음   
1   00988364                          공시 없음   
2   01621183                          공시 없음   
3   00298270                          공시 없음   
4   00402110                          공시 없음   
5   00868705                          공시 없음   
6   00364847                          공시 없음   
7   00

In [80]:
df_latest_disclosures = df_latest_disclosures[['공시 제목', '공시 URL']]
df_latest_disclosures

,공시 제목,공시 URL
0,공시 없음,
1,공시 없음,
2,공시 없음,
3,공시 없음,
4,공시 없음,
5,공시 없음,
6,공시 없음,
7,공시 없음,
8,공시 없음,
9,현금ㆍ현물배당결정,https://dart.fss.or.kr/dsaf001/main.do?rcpNo=2...


In [81]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# 기존 엑셀 파일 경로
file_path = "/content/drive/MyDrive/webcrawling/SGA업계 및 경쟁사 현황_number_and_news.xlsx"
output_path = f"/content/SGA업계 및 경쟁사 현황_{date}.xlsx"

# 엑셀 파일 불러오기
wb = load_workbook(file_path)

# 작업용2 시트 가져오기 (없으면 생성)
if "경쟁사주가" in wb.sheetnames:
    ws = wb["경쟁사주가"]
else:
    ws = wb.create_sheet("경쟁사주가")

# 데이터 추가 시작 셀
start_row = 4  # 데이터 시작 행 번호
start_col = 9  # 데이터 시작 열 번호

# 현재 입력할 행 번호
current_row = start_row

# 데이터 추가
for row_idx, row in df_latest_disclosures.iterrows():
    excel_row_number = current_row


    if excel_row_number in [7,27]:
        current_row += 1  # 현재 행 번호를 1 증가시켜 다음 행으로 이동
    # 공시 제목이 '공시 없음'인 경우 스킵
    if row['공시 제목'] == '공시 없음':
        current_row += 1
        continue

    # 공시 제목 셀 추가 및 하이퍼링크 처리
    title_cell = ws.cell(row=current_row, column=start_col, value=row['공시 제목'])
    title_cell.hyperlink = row['공시 URL']
    title_cell.font = Font(color="0000FF", underline="single")

    # 다음 행으로 이동
    current_row += 1

# 엑셀 파일 저장
wb.save(output_path)
print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")


데이터프레임이 /content/SGA업계 및 경쟁사 현황_2025.01.15.xlsx 파일에 성공적으로 추가되었습니다.
